In [15]:
import cv2
import os
import matplotlib.pyplot as plt
import cachetools
import random 
import numpy as np
import tensorflow as tf
from typing import Tuple

In [11]:

@cachetools.cached(cache={})
def read_video(path: str, frame_index: int = None, min_brightness: int = 10):
    video = cv2.VideoCapture(path)
    if not video.isOpened():
        raise IOError(f"Error opening video file: {path}")
    

    
    frames = []
    def normalize(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC1)
        img = cv2.resize(img, (480, 640), None)
        return img 
    
    while video.isOpened():
        ret, frame = video.read()
        frames.append(frame)
        if not ret:
            break
        if len(frames) >= frame_index and np.average(frame) > min_brightness:
            return normalize(frame)
    if frame_index is None:
        frame_index = len(frames) // 2
        while np.average(frames[frame_index]) < min_brightness:
            frame_index += 1
        return normalize(frames[frame_index]) 
    raise ValueError("frame not found")
REAL_FILES = (
    "1.avi",
    "2.avi",
    "HR_1.avi",
    "HR_4.avi",
)
def read_dir(path: str):
    real_images = []
    spoof_images = []
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)):
            r, s = read_dir(os.path.join(path, file))
            real_images.extend(r)
            spoof_images.extend(s)
        else:
            if file.endswith(".avi"):
                obj = (os.path.join(path, file))
                if file in REAL_FILES:
                    real_images.append(obj)
                    
                else:
                    spoof_images.append(obj)
    return real_images, spoof_images


def show_data(ds, rows, cols):
    random.shuffle(ds)
    for i in range(rows):
        for j in range(cols):
            ind = i * cols + j 
            plt.subplot(rows, cols, ind + 1)
            plt.imshow(ds[ind])
            plt.axis("off")


def load_data(path: str, show = False):
    @cachetools.cached(cache={})
    def inner(path):
        r, s = read_dir(path)
        real = [read_video(i, frame_index=1) for i in r]
        spoof = [read_video(i, frame_index=1) for i in s]
        return real, spoof
    real, spoof = inner(path)
    if show:
        show_data(spoof, 4, 3)
        plt.figure()
        show_data(real, 4, 3)
    ds = [(i, 0) for i in real] + [(i, 1) for i in spoof]
    random.shuffle(ds)
    x = np.array([i[0] for i in ds])
    y = np.array([i[1] for i in ds])
    return x, y

In [12]:
x_train, y_train = load_data("./datasets/train_release")
x_test, y_test = load_data("./datasets/test_release")

In [19]:
# https://openaccess.thecvf.com/content_cvpr_2018/papers/Liu_Learning_Deep_Models_CVPR_2018_paper.pdf
mobilenet = tf.keras.applications.MobileNet(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(640, 480, 3),
    
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)
for l in mobilenet.layers[:len(mobilenet.layers)]:
    l.trainable = False

model = tf.keras.Sequential()
model.add(mobilenet)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(rate = 0.8))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



ValueError: The input must have 3 channels; Received `input_shape=(640, 480, 2)`

In [16]:
def get_fourier(img) -> Tuple[np.ndarray, np.ndarray]:
    # img = cv2.resize(img, dsize=(32, 32), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    fourier = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
    
    fourier_shift = np.fft.fftshift(fourier)

    magnitude = 20*np.log(cv2.magnitude(fourier_shift[:,:,0],fourier_shift[:,:,1]))
    magnitude = cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC1)
 
    phase = cv2.phase(fourier_shift[:,:,0],fourier_shift[:,:,1])
    phase = cv2.normalize(phase, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC1)


    
    return cv2.merge(magnitude, phase)

In [ ]:
def add_salt_pepper(img, rate = 0.2): 
    
    w, h, _ = img.shape 
    noised_image = img.copy()
    white = (255, 255, 255)
    black = (0, 0, 0)
    for i in range(w):
        for j in range(h):
            if random.random() > rate:
                continue
            noised_image[i, j] = white if random.random() > 0.5 else black 
    return noised_image

noised_image = [add_salt_pepper(i) for i in x_train]
blurred_image = [cv2.GaussianBlur(i, (13, 13), 10) for i in x_train]

augmented_x = np.concatenate([x_train, noised_image, blurred_image])
augmented_y = np.concatenate([y_train, y_train, y_train])
ind = list(range(len(augmented_x)))
random.shuffle(ind)
x_train_final = [None for _ in range(len(augmented_x))]
y_train_final = [None for _ in range(len(augmented_x))]
for i in range(len(ind)):
    x_train_final[i] = augmented_x[ind[i]]
    y_train_final[i] = augmented_y[ind[i]]


In [18]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train_final = np.array(x_train_final)
y_train_final = np.array(y_train_final)
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_train.shape, x_train_final.shape)
print(y_train.shape, y_train_final.shape)
print(x_test.shape, y_test.shape)
model.fit(get_fourier(x_train), y_train, epochs=20, validation_data=(get_fourier(x_test),y_test), batch_size=12)


# print(len(x_train_final), len(y_train_final), len(x_train))

(240, 640, 480, 3) (720, 640, 480, 3)
(240,) (720,)
(360, 640, 480, 3) (360,)


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
